# Data Analysis using SparkSession 

Spark Session will provide us with RDD's which are very similar to DataFrames of pandas.

In this notebook we will load a csv file and find the count of customers based on cities.

In the process above, we shall look at some main functions of the dataframes of spark

# Setup

## Load Java Path

In [1]:
# spark is only compatible with Java8 and hence specify its path
import os
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk1.8.0_202'

## Setup SparkContext

In [3]:
import pyspark
from pyspark import SparkContext, SparkConf

conf = pyspark.SparkConf().set('spark.driver.host', '127.0.0.1')

sc = pyspark.SparkContext(master='local', appName='myAppName', conf=conf)

## Setup SparkSession

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession(sc)

# Data Exploration

In [10]:
# Read the csv file into a dataframe and make first entry as header
df = spark.read.option("header", "true").csv('group_by_test.csv')

In [6]:
# the info about df
df

DataFrame[id: string, null_val: string, country: string, region: string, state: string, city: string, date_value: string]

In [9]:
# get top 20 rows to check how the data looks like
df.show()

+------------+--------+-------+------+------------+---------+--------------------+
|          id|null_val|country|region|       state|     city|          date_value|
+------------+--------+-------+------+------------+---------+--------------------+
|Cust_2337980|    null|  India| south|   karnataka|Bangalore|2020-01-10 15:10:...|
|Cust_5132227|    null|  India| north|     Haryana|   Rohtak|2020-01-10 15:10:...|
|Cust_3758463|    null|  India|  west|     Gujarat|Ahmedabad|2020-01-10 15:10:...|
|Cust_5782791|    null|  India| north|UttarPradesh|   Kanpur|2020-01-10 15:10:...|
|Cust_6471658|    null|  India| north|     Haryana|   Rohtak|2020-01-10 15:10:...|
|Cust_1223463|    null|  India|  east|    Jharkand|   Ranchi|2020-01-10 15:10:...|
|Cust_2890272|    null|  India|  east|    Jharkand|   Ranchi|2020-01-10 15:10:...|
|Cust_7912873|    null|  India| south|   karnataka|   Mysore|2020-01-10 15:10:...|
|Cust_6593030|    null|  India| north|UttarPradesh|  Lucknow|2020-01-10 15:10:...|
|Cus

In [11]:
# Get information about all the columns in the dataframe
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- remarks: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date_value: string (nullable = true)



# City related analysis

The advantage of SparkSession is its similarity with SQL

the below are the major commands
1. select --> this is similar to Select in SQL, can be used to pick certain columns from the dataframe
2. filter --> this is similar to where clause in SQL, can be used to filter out based on condition
3. groupBy--> exaclty same as SQL Group-By clause

In [8]:
# Check the top 4 rows and get only city values
df.select('city').head(4)

[Row(city='Bangalore'),
 Row(city='Rohtak'),
 Row(city='Ahmedabad'),
 Row(city='Kanpur')]

In [12]:
# Get states and countries in seperate df
country_state_map=df.select(df['country'],df['state'])

In [13]:
# view the results
country_state_map.show()

+-------+------------+
|country|       state|
+-------+------------+
|  India|   karnataka|
|  India|     Haryana|
|  India|     Gujarat|
|  India|UttarPradesh|
|  India|     Haryana|
|  India|    Jharkand|
|  India|    Jharkand|
|  India|   karnataka|
|  India|UttarPradesh|
|  India|     Haryana|
|  India|     Gujarat|
|  India|UttarPradesh|
|  India|  tamil nadu|
|  India|   karnataka|
|  India|   karnataka|
|  India| Maharashtra|
|  India|     Gujarat|
|  India| Maharashtra|
|  India|  tamil nadu|
|  India|     Haryana|
+-------+------------+
only showing top 20 rows



In [14]:
# Check the top 5 rows where city is "Bangaloreabs"
df.filter(df['city']=='Bangalore').head(5)

[Row(id='Cust_2337980', remarks=None, country='India', region='south', state='karnataka', city='Bangalore', date_value='2020-01-10 15:10:38.562000'),
 Row(id='Cust_3934433', remarks=None, country='India', region='south', state='karnataka', city='Bangalore', date_value='2020-01-10 15:10:38.562000'),
 Row(id='Cust_9294147', remarks=None, country='India', region='south', state='karnataka', city='Bangalore', date_value='2020-01-10 15:10:38.562000'),
 Row(id='Cust_2772376', remarks=None, country='India', region='south', state='karnataka', city='Bangalore', date_value='2020-01-10 15:10:38.562000'),
 Row(id='Cust_4252429', remarks=None, country='India', region='south', state='karnataka', city='Bangalore', date_value='2020-01-10 15:10:38.562000')]

In [16]:
# Get count of rows per each cutomer and view results
df.groupBy(df['city']).count().show()

+----------+-----+
|      city|count|
+----------+-----+
| Bangalore| 6153|
|    Mysore| 6255|
|  Vadodara| 6307|
|   Lucknow| 6258|
|   Chennai| 6127|
|    Ranchi| 6250|
|Jamshedpur| 6266|
|    Mumbai| 6150|
| Ahmedabad| 6216|
|   Kolkata| 6204|
|    Rohtak| 6403|
|   Gurgaon| 6205|
|      Pune| 6327|
|    Kanpur| 6323|
|Darjeeling| 6285|
|  Madhurai| 6271|
+----------+-----+



# SQL queries in SPARK

We can write SQL queries and execute them in spark directly without using HIVE,

This can be achived by using sparksql.

We will make a view from the dataframe and then run the queries

In [17]:
df.createOrReplaceTempView('cust')

In [20]:
city_counts_df=spark.sql('select city,count(1) from cust group by city')

In [21]:
city_counts_df.show()

+----------+--------+
|      city|count(1)|
+----------+--------+
| Bangalore|    6153|
|    Mysore|    6255|
|  Vadodara|    6307|
|   Lucknow|    6258|
|   Chennai|    6127|
|    Ranchi|    6250|
|Jamshedpur|    6266|
|    Mumbai|    6150|
| Ahmedabad|    6216|
|   Kolkata|    6204|
|    Rohtak|    6403|
|   Gurgaon|    6205|
|      Pune|    6327|
|    Kanpur|    6323|
|Darjeeling|    6285|
|  Madhurai|    6271|
+----------+--------+



In [22]:
unique_city_df=spark.sql('select distinct city from cust')

In [23]:
unique_city_df.show()

+----------+
|      city|
+----------+
| Bangalore|
|    Mysore|
|  Vadodara|
|   Lucknow|
|   Chennai|
|    Ranchi|
|Jamshedpur|
|    Mumbai|
| Ahmedabad|
|   Kolkata|
|    Rohtak|
|   Gurgaon|
|      Pune|
|    Kanpur|
|Darjeeling|
|  Madhurai|
+----------+



# Halt SparkContext


In [24]:
sc.stop()

# Summary

1. SparkSession will be used to make sql like operations on DF objects
2. We have majorly Select,Filter and GroupBY
3. createOrReplaceTempView can be used to make a view on a df
4. using spark.sql we can run sql queries on views that was created from df